In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy import stats
import seaborn as sns
import os
import os.path as osp
import sys
import pickle
import joblib
from collections import Counter
from itertools import product
import torch
import pdb
import random
import tables
from sklearn.linear_model import LogisticRegression, LinearRegression
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report, average_precision_score,\
balanced_accuracy_score
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.calibration import CalibratedClassifierCV
import wandb
from wandb.lightgbm import wandb_callback, log_summary
from dill.source import getsource
from dill import detect
import functools

In [ ]:
print(np.__version__)
print(pd.__version__)

### set the seeds and change to current directory + set the output directory

In [ ]:
SEED=42
np.random.seed(SEED)
os.environ['USER_PATH']='/share/pierson/selective_labels_data/hirid_data_analysis/richras_dir/learning_from_doctor_and_patient/'
os.environ['OUT_PATH']='/share/pierson/selective_labels_data/hirid_data_analysis/richras_dir/learning_from_doctor_and_patient/output_directory'

In [ ]:
os.chdir('/share/pierson/selective_labels_data/hirid_data_analysis/richras_dir/learning_from_doctor_and_patient/')
from AnalysisFuncs import plotCorr, getPred_fromProb, getMetrics, getCorrMedical, getGroundTruth, getURange
from AnalysisFuncs import getResiduals, plotDistributionProbs, plotCorr_w_Unobs
from AnalysisFuncs import saveFile, loadFile, getTrainTestIdx, getLabels

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
processed_data_path=osp.join(os.environ.get('USER_PATH'), 'HIRID_Repo', 'logs', 'benchmark_exp')

In [ ]:
test_X = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
                '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured', '1111'), 'test_rep.pkl')
test_y_T = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
                '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured', '1111'), 'test_label.pkl')
test_ids_T = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
        '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured', '1111'), 'test_patient_ids.pkl')

test_X_D_given_T = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
    '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Above_Threshold', '1111'), 'test_rep.pkl')
test_y_D_given_T = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
    '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Above_Threshold', '1111'), 'test_label.pkl')
test_ids_D_given_T = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
    '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Above_Threshold', '1111'), 'test_patient_ids.pkl')

test_X_D_and_T = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
    '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured_And_Disease', '1111'), 'test_rep.pkl')
test_y_D_and_T = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
    '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured_And_Disease', '1111'), 'test_label.pkl')
test_ids_D_and_T = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
    '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured_And_Disease', '1111'), 'test_patient_ids.pkl')

In [ ]:
test_X_wo_cutoff = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2', 
                '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured', '1111'), 'test_rep.pkl')
test_y_T_wo_cutoff = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2', 
                '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured', '1111'), 'test_label.pkl')
test_ids_T_wo_cutoff = loadFile(osp.join(processed_data_path,'LGBM_w_feat', 
        '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured', '1111'), 'test_patient_ids.pkl')

test_X_D_given_T_wo_cutoff = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2', 
    '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Above_Threshold', '1111'), 'test_rep.pkl')
test_y_D_given_T_wo_cutoff = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2', 
    '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Above_Threshold', '1111'), 'test_label.pkl')
test_ids_D_given_T_wo_cutoff = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2', 
    '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Above_Threshold', '1111'), 'test_patient_ids.pkl')

test_X_D_and_T_wo_cutoff = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2', 
    '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured_And_Disease', '1111'), 'test_rep.pkl')
test_y_D_and_T_wo_cutoff = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2', 
    '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured_And_Disease', '1111'), 'test_label.pkl')
test_ids_D_and_T_wo_cutoff = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2', 
    '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured_And_Disease', '1111'), 'test_patient_ids.pkl')

In [ ]:
val_X = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
                '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured'), 'val_rep.pkl')
val_y_T = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
                '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured'), 'val_labels.pkl')
val_ids_T = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
        '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured'), 'val_pids.pkl')

val_X_D_given_T = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
        '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Above_Threshold'), 'val_rep.pkl')
val_y_D_given_T = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
        '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Above_Threshold'), 'val_labels.pkl')
val_ids_D_given_T = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
        '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Above_Threshold'), 'val_pids.pkl')

val_X_D_and_T = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
        '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured_And_Disease'), 'val_rep.pkl')
val_y_D_and_T = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
        '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured_And_Disease'), 'val_labels.pkl')
val_ids_D_and_T = loadFile(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
        '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured_And_Disease'), 'val_pids.pkl')

In [ ]:
#load trained models
LGBM_w_feat_v2_cutoff_T = joblib.load(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
    '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured', '1111', 'model.pkl'))
LGBM_w_feat_v2_cutoff_D_given_T = joblib.load(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
    '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Above_Threshold', '1111', 'model.pkl'))
LGBM_w_feat_v2_cutoff_D_and_T = joblib.load(osp.join(processed_data_path,'LGBM_w_feat_v2_cutoff_T', 
    '_depth_7_subsample-data_1.0_subsample-feat_1.0', 'Lactate_Measured_And_Disease', '1111', 'model.pkl'))
LGBM_D_given_T_ipw = loadFile(osp.join(processed_data_path, 'LGBM_w_feat_v2_cutoff_T', 'predict_D_given_T_ipw'),
         'LGBM_D_given_T_ipw.pkl')
LGBM_D_pseudo = loadFile(osp.join(processed_data_path, 'LGBM_w_feat_v2_cutoff_T','predict_D_pseudo'), 
         'LGBM_D_pseudo.pkl')

In [ ]:
models=['LGBM']
tasks=['T', 'D|T', 'D|T_ipw', 'D_and_T','D_pseudo']
clf_list=[LGBM_w_feat_v2_cutoff_T, LGBM_w_feat_v2_cutoff_D_given_T, LGBM_D_given_T_ipw,
          LGBM_w_feat_v2_cutoff_D_and_T, LGBM_D_pseudo]
clf_dict={}
i=0
for t in tasks:
    clf_dict[t]={}
    for m in models:
        clf_dict[t][m]=clf_list[i]
        i+=1
dict_df_labels={}
dict_df_probs={}
dict_df_ids={}
dict_models={}
probs_path=processed_data_path
alpha=0.1
figsize1=(5,50)
figsize2=(10,10)
df_pp=pd.DataFrame({'AUC' : [],'PR':[],'BalancedAcc':[],'modelName':[],'rowName':[]})

In [ ]:
processed_data_path

In [ ]:
# make dataframe of labels by matching it with the patient ids
df_labels = pd.DataFrame([])
df_labels['pids'] = np.concatenate((test_ids_T, val_ids_T))
df_labels['T'] = np.concatenate((test_y_T, val_y_T))
test_y_D_given_T_all = getLabels(test_y_D_given_T, test_ids_D_given_T, test_ids_T)
val_y_D_given_T_all = getLabels(val_y_D_given_T, val_ids_D_given_T, val_ids_T)

test_y_D_and_T_all = getLabels(test_y_D_and_T, test_ids_D_and_T, test_ids_T)
val_y_D_and_T_all = getLabels(val_y_D_and_T, val_ids_D_and_T, val_ids_T)

df_labels['D'] = np.concatenate((test_y_D_given_T_all, val_y_D_given_T_all))
df_labels['D_and_T'] = np.concatenate((test_y_D_and_T_all, val_y_D_and_T_all))

test_idxs = df_labels.index[df_labels['pids'].isin(test_ids_T)].values
cal_idxs = df_labels.index[df_labels['pids'].isin(val_ids_T)].values

In [ ]:
df_labels

In [ ]:
df_labels_wo_cutoff = pd.DataFrame([])
df_labels_wo_cutoff['pids'] = test_ids_T_wo_cutoff
df_labels_wo_cutoff['T'] = test_y_T_wo_cutoff
test_y_D_given_T_wo_cutoff_all = getLabels(test_y_D_given_T_wo_cutoff, test_ids_D_given_T_wo_cutoff,
                                           test_ids_T_wo_cutoff)


test_y_D_and_T_wo_cutoff_all = getLabels(test_y_D_and_T_wo_cutoff, test_ids_D_and_T_wo_cutoff,
                                         test_ids_T_wo_cutoff)

df_labels_wo_cutoff['D|T'] = test_y_D_given_T_wo_cutoff_all
df_labels_wo_cutoff['D_and_T'] = test_y_D_and_T_wo_cutoff_all

test_idxs_wo_cutoff = df_labels_wo_cutoff.index[df_labels_wo_cutoff['pids'].isin(test_ids_T_wo_cutoff)].values


In [ ]:
df_labels_wo_cutoff

In [ ]:
test_idxs

In [ ]:
cal_idxs

In [ ]:
probs_path

In [ ]:
calibrate_method = "sigmoid"

In [ ]:
dict_df_labels, dict_df_probs, dict_models, df_pp = getCorrMedical(models, tasks, test_X.copy(), clf_dict,
        df_labels.copy(), 
        test_idxs.copy(), dict_df_labels, dict_df_probs, dict_models, df_pp, probs_path, calibrate=True, 
        figsize1=figsize1, figsize2=figsize2, alpha=alpha, cal_idxs=cal_idxs, cal_X=val_X,
                                                                   calibrate_method=calibrate_method)

In [ ]:
corr_mat=dict_df_probs['LGBM'].corr(method='pearson')
np.linalg.eigvals(corr_mat)

In [ ]:
title='medical correlation matrix'
corr_method="spearman"
figsize=(9, 10)
top_adjust=0.9
title_en=False
plotCorr(models, dict_df_probs, title, corr_method, figsize=figsize, top_adjust=top_adjust, 
         title_en=title_en, savefig_path=processed_data_path)

In [ ]:
assert not np.isnan(dict_df_probs['LGBM']['T'].values).any()

In [ ]:
title=r'Correlation with $P(D=1|X)$' '\n' r'when $u(x)=sigmoid(\beta X)$'
# models=['LogisticRegression', 'LGBM']
models=['LGBM']
# tasks=['T', 'D|T', 'D|T_ipw', 'D_and_T', 'product_T_D_given_T', 'D_pseudo']
tasks=['T', 'D|T', 'D|T_ipw', 'D_and_T','D_pseudo']
figsize=(7,5)
# figsize=(10,7)
top_adjust=0.9
# tasks=['T', 'D|T']
corr_method=stats.pearsonr
title_en=True
plotCorr_w_Unobs(dict_df_probs, models, title, tasks, alpha=['sigmoid'],X=test_X, corr_method=corr_method,
                 figsize=figsize, top_adjust=top_adjust, title_en=title_en)

In [ ]:
title=r'Correlation with $P(D=1|X)$' '\n' r'when $u(x)=\alpha P(D=1|T=1,X)$'
# models=['LogisticRegression', 'LGBM']
models=['LGBM']
# tasks=['T', 'D|T', 'D|T_ipw', 'D_and_T', 'product_T_D_given_T', 'D_pseudo']
tasks=['T', 'D|T', 'D|T_ipw', 'D_and_T','D_pseudo']
figsize=(7,5)
# figsize=(10,7)
top_adjust=0.9
# tasks=['T', 'D|T']
corr_method=stats.pearsonr
loc="center left"
# loc="lower left"
title_en=False
plotCorr_w_Unobs(dict_df_probs, models, title, tasks, alpha=np.arange(0,1.1,0.1), corr_method=corr_method,
                 figsize=figsize, top_adjust=top_adjust, title_en=title_en, loc=loc)

In [ ]:
title=r'Correlation with $P(D=1|X)$' '\n' r'when $u(x)=\alpha$ is constant'
models=['LGBM']
# tasks=['T', 'D|T']
corr_method=stats.pearsonr
tasks=['T', 'D|T', 'D|T_ipw', 'D_and_T','D_pseudo']
figsize=(7,5)
# figsize=(10,7)
# loc="center left"
loc="lower left"
title_en=False
plotCorr_w_Unobs(dict_df_probs, models, title, tasks, corr_method=corr_method,
                 figsize=figsize, loc=loc, title_en=title_en, custome_ticker=True)

In [ ]:
df_probs_stats=pd.DataFrame({'model' : [],'task':[],'mean':[],'std':[], 'min':[], 'max':[]})
tasks=['T','D|T']
for m in models:
    for t in tasks:
        df_probs_stats=df_probs_stats.append({'model' : m,'task':t,'mean':dict_df_probs[m][t].mean(),
                               'std':dict_df_probs[m][t].std(), 'min':dict_df_probs[m][t].min(),
                                'max': dict_df_probs[m][t].max()}, ignore_index=True)
        print(f" mean , std, min. for model {m} and task {t}: {dict_df_probs[m][t].mean():.3f}, {dict_df_probs[m][t].std():.3f}, {dict_df_probs[m][t].min():.3f}")

In [ ]:
df_probs_stats

In [ ]:
title='Distribution of estimated probs'
models=['LGBM']
tasks=['D|T']
figsize=(10,7)
plotDistributionProbs(dict_df_probs, models, title, tasks, figsize=figsize)

In [ ]:
title='Distribution of estimated probs'
models=['LGBM']
tasks=['T']
figsize=(10,7)
plotDistributionProbs(dict_df_probs, models, title, tasks, figsize=figsize)

In [ ]:
for m in models:
    print(f"for model :{m}")
    getResiduals(dict_df_probs[m]['D_and_T'], dict_df_probs[m]['D|T'], dict_df_probs[m]['T'])

In [ ]:
dict_df_labels['LGBM']

In [ ]:
df_ids_test = pd.DataFrame([])
df_labels_copy = df_labels.loc[test_idxs].copy()
all_pids = df_labels_copy['pids']
df_ids_test['T'] = all_pids.copy()
df_ids_test['D|T'] = np.nan
df_ids_test.loc[~pd.isna(df_labels_copy['D']),['D|T']] = all_pids.loc[~pd.isna(df_labels_copy['D'])]
df_ids_test['D_and_T'] = np.nan
df_ids_test.loc[~pd.isna(df_labels_copy['D']),['D_and_T']] = all_pids.loc[~pd.isna(df_labels_copy['D'])]
print(df_ids_test)


In [ ]:
df_labels_copy

In [ ]:
df_ids_test

In [ ]:
df_ids_test_wo_cutoff = pd.DataFrame([])
df_labels_wo_cutoff_copy = df_labels_wo_cutoff.loc[test_idxs_wo_cutoff].copy()
all_pids_wo_cutoff = df_labels_wo_cutoff_copy['pids']
df_ids_test_wo_cutoff['T'] = all_pids_wo_cutoff.copy()
df_ids_test_wo_cutoff['D|T'] = np.nan
df_ids_test_wo_cutoff.loc[~pd.isna(df_labels_wo_cutoff_copy['D|T']),['D|T']] = \
                            all_pids_wo_cutoff.loc[~pd.isna(df_labels_wo_cutoff_copy['D|T'])]
print(df_ids_test_wo_cutoff)

In [ ]:
df_labels_wo_cutoff

In [ ]:
endpoints = []
patients_already_seen = set()
for i in range(250):
    endpoints.append(pd.read_parquet(
        '/share/pierson/selective_labels_data/hirid_data_analysis/richras_dir/learning_from_doctor_and_patient/output_directory/endpoints_w_lac/batch_%i.parquet' % i))
    assert len(patients_already_seen.intersection(set(endpoints[-1]['patientid']))) == 0 # This assert checks that 
    # only new patient ids are added, i.e patientid is a unique field
    old_len = len(patients_already_seen)
    patients_already_seen = patients_already_seen.union(set(endpoints[-1]['patientid']))
    # print(len(patients_already_seen) - old_len, 'new patients')
endpoints = pd.concat(endpoints)
print(endpoints.info())

In [ ]:
general_table = pd.read_parquet('/share/pierson/selective_labels_data/hirid_data_analysis/output_directory/general_table_extended.parquet')
mort_status = general_table[['patientid', 'discharge_status']].copy()
mortality_dict={"alive":0, "dead":1}
mort_status['status']=mort_status['discharge_status'].map(mortality_dict)
length_staySeries = endpoints.groupby(['patientid'])['rel_datetime'].count().reset_index()
#first column is patientid, second is relative length of stay
length_stay = np.array(length_staySeries.values)
print(length_stay, length_stay.shape)
mort_status_rel_stay = pd.merge(mort_status, pd.DataFrame(length_staySeries), on='patientid')
mort_status_rel_stay

In [ ]:
# long term outcomes

def set_axis_style(ax, labels, title, xlabel, ylabel):
    ax.xaxis.set_tick_params(direction='out')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xticks(np.arange(1, len(labels) + 1), labels=labels)
    ax.set_xlim(0.25, len(labels) + 0.75)
    ax.set_xlabel(xlabel+" (Binary Label)")
    ax.set_title(title)
    ax.set_ylabel(ylabel)    

def getCommonPids(pid_ls1, pid_ls2):
    ret_pids1, ret_pids2=[],[]
    intersect_pids=np.intersect1d(pid_ls1, pid_ls2)
    for pid in intersect_pids:
        ret_pids1.append(np.argwhere(pid_ls1==pid).item())
        ret_pids2.append(np.argwhere(pid_ls2==pid).item())
    assert len(intersect_pids)==len(ret_pids1)==len(ret_pids2)
    assert len(intersect_pids)>1
    return intersect_pids, ret_pids1, ret_pids2

def getLongerOutcomesMetrics(modelNames, tasks, estNames, outcome_D, title, xLabel, yLabel, df_metrics, dict_df_ids_cutoff, dict_df_labels_cutoff, dict_df_probs_cutoff, dict_df_labels, dict_df_ids, **kwargs):
    """ Get metrics between task A and B and longer time outcomes
    args:
        estimators[0][mName]={}, estimators[1][mName]={}
    """
    fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(15, 9), sharey=True)
    i,j=0,0
        
    outcome_D_name=kwargs.get('outcome_D_name')
    ipw=kwargs.get('ipw')
    for mName in modelNames:
        
        idx_untested=np.where(dict_df_labels_cutoff['T']==0)[0]
        
        T_labels = np.array(dict_df_labels['T'].values.tolist())
        T_ids = np.array(dict_df_ids['T'].values.tolist())
        
        untested_T_cutoff_ids = np.array(dict_df_ids_cutoff['T'].values.tolist())
        
        T_labels = getLabels(T_labels, T_ids, untested_T_cutoff_ids)
        untested_T_labels=T_labels[idx_untested][~np.isnan(T_labels[idx_untested])]
        
        untested_T_probs = np.array(dict_df_probs_cutoff['T'].values.tolist())[idx_untested]
        untested_T_probs=untested_T_probs[~np.isnan(T_labels[idx_untested])]
        untested_D_probs = np.array(dict_df_probs_cutoff[str(tasks[1])].values.tolist())[idx_untested]
        untested_D_probs=untested_D_probs[~np.isnan(T_labels[idx_untested])] 
        
        assert len(untested_T_labels)==len(untested_T_probs)
        print(f"labels for untested T=0 pop for model {str(mName)}, :{len(untested_T_probs)}")
        print(f"correlation 1x1:"
        f"{stats.spearmanr(untested_T_labels, untested_T_probs)}")
        auc_1_1, pr_1_1, ba_1_1=getMetrics(untested_T_labels, untested_T_probs)[:3]
        cr_1_1=getMetrics(untested_T_labels, untested_T_probs)[-1]
        print(f"PR, balanced accuracy metrics for 1x1 :{str(mName)}, {pr_1_1},{ba_1_1} \n {cr_1_1}")
        print(f"auc score 1x1:{auc_1_1:.4f}")
        
        auc_2_1, pr_2_1, ba_2_1=getMetrics(untested_T_labels, untested_D_probs)[:3]
        cr_2_1=getMetrics(untested_T_labels, untested_D_probs)[-1]
        print(f"correlation 2x1:{stats.spearmanr(untested_T_labels, untested_D_probs)}")
        print(f"PR, balanced accuracy metrics for 2x1 :{str(mName)}, {pr_2_1},{ba_2_1} \n {cr_2_1}")
        print(f"auc score 2x1:{auc_2_1:.4f}")
        
        df_metrics=df_metrics.append({'AUC' :auc_1_1 ,'PR':pr_1_1,'BalancedAcc':ba_1_1, 'modelName':mName,'rowName':estNames[0],'colName':'T'},ignore_index=True)
        df_metrics=df_metrics.append({'AUC' :auc_2_1 ,'PR':pr_2_1,'BalancedAcc':ba_2_1,'modelName':mName,'rowName':estNames[1],'colName':'T'},ignore_index=True)

        ax[i,j].violinplot([untested_T_probs[untested_T_labels==i] for i in [0,1]], 
                           showmeans=False, showmedians=True,showextrema=True)
        ax[i+1,j].violinplot([untested_D_probs[untested_T_labels==i] for i in [0,1]],
                             showmeans=False, showmedians=True,showextrema=True)
        
        #2,2 and 1,2
        idx_untested_D_given_T = np.where((dict_df_labels_cutoff['T']==0) & 
                                            (T_labels==1)[0])
                
        T_ids = np.array(dict_df_ids['T'].values.tolist())
#         D_given_T_labels = np.array(dict_df_labels[str(tasks[1])].values.tolist())
#         D_given_T_labels = getLabels(D_given_T_labels, T_ids, untested_T_cutoff_ids)
#         untested_D_given_T_labels = D_given_T_labels[idx_untested_D_given_T][~np.isnan(D_given_T_labels[idx_untested_D_given_T])]
        
        outcome_D_labels = np.array(dict_df_labels[outcome_D].values.tolist())
        outcome_D_labels = getLabels(outcome_D_labels, T_ids, untested_T_cutoff_ids)
        untested_outcome_D_labels=outcome_D_labels[idx_untested_D_given_T][~np.isnan(outcome_D_labels[idx_untested_D_given_T])]
        
        untested_T_probs = np.array(dict_df_probs_cutoff['T'].values.tolist())[idx_untested_D_given_T]
        untested_T_probs=untested_T_probs[~np.isnan(outcome_D_labels[idx_untested_D_given_T])]
        untested_D_probs = np.array(dict_df_probs_cutoff[str(tasks[1])].values.tolist())[idx_untested_D_given_T]
        untested_D_probs=untested_D_probs[~np.isnan(outcome_D_labels[idx_untested_D_given_T])]
        
#         print(f"labels for untested T=0 after 1 hour and D=1 pop for model {str(mName)}, :{len(untested_D_given_T_labels)}")
        print(f"correlation 1x2:"
        f"{stats.spearmanr(untested_outcome_D_labels, untested_T_probs)}")
        
        print(f"correlation 2x2:"
        f"{stats.spearmanr(untested_outcome_D_labels, untested_D_probs)}")
        auc_1_2, pr_1_2, ba_1_2=getMetrics(untested_outcome_D_labels, untested_T_probs)[:3]
        cr_1_2=getMetrics(untested_outcome_D_labels, untested_T_probs)[-1]
        print(f"auc score 1x2:{auc_1_2:.4f}")
        
        auc_2_2, pr_2_2, ba_2_2=getMetrics(untested_outcome_D_labels, untested_D_probs)[:3]
        cr_2_2=getMetrics(untested_outcome_D_labels, untested_D_probs)[-1]
        print(f"auc score 2x2:{auc_2_2:.4f}")
        print(f"PR, balanced accuracy metrics for 1x2 :{str(mName)}, {pr_1_2},{ba_1_2} \n {cr_1_2}")

        print(f"PR, balanced accuracy metrics for 2x2 :{str(mName)}, {pr_2_2},{ba_2_2} \n {cr_2_2}")

        df_metrics=df_metrics.append({'AUC' :auc_1_2 ,'PR':pr_1_2,'BalancedAcc':ba_1_2,'modelName':mName,'rowName':estNames[0],'colName':f"{outcome_D_name}"},ignore_index=True)
        df_metrics=df_metrics.append({'AUC' :auc_2_2 ,'PR':pr_2_2,'BalancedAcc':ba_2_2,'modelName':mName,'rowName':estNames[1],'colName':f"{outcome_D_name}"},ignore_index=True)

        ax[i,j+1].violinplot([untested_T_probs[untested_outcome_D_labels==i] for i in [0,1]], 
                             showmeans=False, showmedians=True, showextrema=True)
        ax[i+1,j+1].violinplot([untested_D_probs[untested_outcome_D_labels==i] for i in [0,1]],
                               showmeans=False, showmedians=True, showextrema=True)

        #1,3 and 2,3

        common_mort_pids, idx_labels_mort, idx_probs_mort =\
                getCommonPids(mort_status_rel_stay.loc[(~pd.isnull(mort_status_rel_stay['discharge_status']))\
                            & (mort_status_rel_stay['rel_datetime']>12)]['patientid'].values.tolist(),
                untested_T_cutoff_ids[idx_untested])
        mort_labels=np.array(mort_status_rel_stay.loc[(~pd.isnull(mort_status_rel_stay['discharge_status']))\
                            & (mort_status_rel_stay['rel_datetime']>12)]['status'].values.tolist())[idx_labels_mort]

        assert np.isnan(mort_labels).sum()==0
        
        untested_T_probs = np.array(dict_df_probs_cutoff['T'].values.tolist())[idx_untested]
        untested_D_probs = np.array(dict_df_probs_cutoff[str(tasks[1])].values.tolist())[idx_untested]

        print(f"labels for untested T=0 pop for model {str(mName)} and mort status, :{len(common_mort_pids)}")

        print(f"correlation 1x3:"
        f"{stats.spearmanr(mort_labels, untested_T_probs[idx_probs_mort])}")
        auc_1_3, pr_1_3, ba_1_3=getMetrics(mort_labels, untested_T_probs[idx_probs_mort])[:3]
        cr_1_3=getMetrics(mort_labels, untested_T_probs[idx_probs_mort])[-1]
        print(f"PR, balanced accuracy metrics for 1x3 :{str(mName)}, {pr_1_3},{ba_1_3} \n {cr_1_3}")
        print(f"correlation 2x3:"
        f"{stats.spearmanr(mort_labels, untested_D_probs[idx_probs_mort])}")
        auc_2_3, pr_2_3, ba_2_3=getMetrics(mort_labels, untested_D_probs[idx_probs_mort])[:3]
        cr_2_3=getMetrics(mort_labels, untested_D_probs[idx_probs_mort])[-1]
        print(f"PR, balanced accuracy metrics for 2x3 :{str(mName)}, {pr_2_3},{ba_2_3} \n {cr_2_3}")
        print(f"auc score 1x3:{auc_1_3:.4f}")
        print(f"auc score 2x3:{auc_2_3:.4f}")
        df_metrics=df_metrics.append({'AUC' :auc_1_3 ,'PR':pr_1_3,'BalancedAcc':ba_1_3,'modelName':mName,'rowName':estNames[0],'colName':'Mortality'},ignore_index=True)
        df_metrics=df_metrics.append({'AUC' :auc_2_3 ,'PR':pr_2_3,'BalancedAcc':ba_2_3,'modelName':mName,'rowName':estNames[1],'colName':'Mortality'},ignore_index=True)

        ax[i,j+2].violinplot([untested_T_probs[idx_probs_mort][mort_labels==i] for i in [0,1]], 
                             showmeans=False, showmedians=True, showextrema=True)
        ax[i+1,j+2].violinplot([untested_D_probs[idx_probs_mort][mort_labels==i] for i in [0,1]],
                               showmeans=False, showmedians=True, showextrema=True)
        print(f"len of alive:{len([untested_T_probs[idx_probs_mort][mort_labels==i] for i in [0]][0])}")
        print(f"len of dead:{len([untested_T_probs[idx_probs_mort][mort_labels==i] for i in [1]][0])}")
        # set style for the axes
        labels = ['0', '1']
        
        for k,a in enumerate([ax[0,0], ax[0,1], ax[0,2], ax[1,0], ax[1,1], ax[1,2]]):
            set_axis_style(a, labels, title[k], xLabel[k], yLabel[k])
        fig.tight_layout()
        plt.show()
    plt.close()
    return df_metrics

In [ ]:
dict_df_probs['LGBM']

In [ ]:
df_labels_wo_cutoff

In [ ]:
df_metrics=pd.DataFrame({'AUC' : [],'PR':[],'BalancedAcc':[],'modelName':[],'rowName':[],'colName':[]})
modelNames=['LGBM']
tasks=['T', 'D|T']
estNames=[r'$p(T_{t=1hr}=1|X)$', r'$p(D_{t=1hr}=1|T_{t=1hr}=1,X)$']
title=[r'$p(T_{t=1 hr}) \quad X \quad T_{ever}$', r'$p(T_{t=1 hr}) \quad X \quad D|T_{ever}$',\
               r'$p(T_{t=1 hr}) \quad X \quad MortStatus$', r'$p(D_{t=1 hr}|T_{t=1 hr}) \quad X \quad T_{ever}$',\
               r'$p(D_{t=1 hr}|T_{t=1 hr}) \quad X \quad D|T_{ever}$', r'$p(D_{t=1 hr}|T_{t=1 hr}) \quad X \quad MortStatus$']
yLabel=[r'$p(T_{t=1 hr})$', r'$p(T_{t=1 hr})$',\
       r'$p(T_{t=1 hr})$', r'$p(D_{t=1 hr}|T_{t=1 hr})$',\
       r'$p(D_{t=1 hr}|T_{t=1 hr}) $', r'$p(D_{t=1 hr}|T_{t=1 hr})$']
xLabel=[r'$T_{ever}$', r'$D|T_{ever}$',\
       r'$MortStatus$', r'$T_{ever}$',\
       r'$D|T_{ever}$', r'$MortStatus$']
outcome_D='D|T'
outcome_D_name='D|T'
df_metrics=getLongerOutcomesMetrics(modelNames, tasks, estNames, outcome_D, title, xLabel, yLabel,
df_metrics, df_ids_test, dict_df_labels['LGBM'], dict_df_probs['LGBM'], df_labels_wo_cutoff, 
                                    df_ids_test_wo_cutoff, outcome_D_name=outcome_D_name)

In [ ]:
modelNames=['LGBM']
tasks=['T', 'D_and_T']
estNames=[r'$p(T_{t=1hr}=1|X)$', r'$p(D_{t=1hr}=1,T_{t=1hr}=1|X)$']
title=[r'$p(T_{t=1 hr}) \quad X \quad T_{ever}$', r'$p(T_{t=1 hr}) \quad X \quad D,T_{ever}$',\
               r'$p(T_{t=1 hr}) \quad X \quad MortStatus$', r'$p(D_{t=1 hr},T_{t=1 hr}) \quad X \quad T_{ever}$',\
               r'$p(D_{t=1 hr},T_{t=1 hr}) \quad X \quad D|T_{ever}$', r'$p(D_{t=1 hr},T_{t=1 hr}) \quad X \quad MortStatus$']
yLabel=[r'$p(T_{t=1 hr})$', r'$p(T_{t=1 hr})$',\
       r'$p(T_{t=1 hr})$', r'$p(D_{t=1 hr},T_{t=1 hr})$',\
       r'$p(D_{t=1 hr},T_{t=1 hr}) $', r'$p(D_{t=1 hr},T_{t=1 hr})$']
xLabel=[r'$T_{ever}$', r'$D,T_{ever}$',\
       r'$MortStatus$', r'$T_{ever}$',\
       r'$D,T_{ever}$', r'$MortStatus$']
outcome_D='D_and_T'
outcome_D_name='D,T'
df_metrics=getLongerOutcomesMetrics(modelNames, tasks, estNames, outcome_D, title, xLabel, yLabel,
df_metrics, df_ids_test, dict_df_labels['LGBM'], dict_df_probs['LGBM'], df_labels_wo_cutoff, 
                                    df_ids_test_wo_cutoff, outcome_D_name=outcome_D_name)

In [ ]:
modelNames=['LGBM']
tasks=['T', 'D_and_T']
estNames=[r'$p(T_{t=1hr}=1|X)$', r'$p(D_{t=1hr}=1,T_{t=1hr}=1|X)$']
outcome_D='D|T'
outcome_D_name='D|T'
title=[r'$p(T_{t=1 hr}) \quad X \quad T_{ever}$', r'$p(T_{t=1 hr}) \quad X \quad D|T_{ever}$',\
               r'$p(T_{t=1 hr}) \quad X \quad MortStatus$', r'$p(D_{t=1 hr},T_{t=1 hr}) \quad X \quad T_{ever}$',\
               r'$p(D_{t=1 hr},T_{t=1 hr}) \quad X \quad D|T_{ever}$', r'$p(D_{t=1 hr},T_{t=1 hr}) \quad X \quad MortStatus$']
yLabel=[r'$p(T_{t=1 hr})$', r'$p(T_{t=1 hr})$',\
       r'$p(T_{t=1 hr})$', r'$p(D_{t=1 hr},T_{t=1 hr})$',\
       r'$p(D_{t=1 hr},T_{t=1 hr}) $', r'$p(D_{t=1 hr},T_{t=1 hr})$']
xLabel=[r'$T_{ever}$', r'$D|T_{ever}$',\
       r'$MortStatus$', r'$T_{ever}$',\
       r'$D|T_{ever}$', r'$MortStatus$']
df_metrics=getLongerOutcomesMetrics(modelNames, tasks, estNames, outcome_D, title, xLabel, yLabel,
df_metrics, df_ids_test, dict_df_labels['LGBM'], dict_df_probs['LGBM'], df_labels_wo_cutoff, 
                                    df_ids_test_wo_cutoff, outcome_D_name=outcome_D_name)

In [ ]:
modelNames=['LGBM']
tasks=['T', 'D|T_ipw']
estNames=[r'$p(T_{t=1hr}=1|X)$', r'$IPW_{t=1hr}$']
outcome_D='D|T'
outcome_D_name='D|T'
title=[r'$p(T_{t=1 hr}) \quad X \quad T_{ever}$', r'$p(T_{t=1 hr}) \quad X \quad D|T_{ever}$',\
               r'$p(T_{t=1 hr}) \quad X \quad MortStatus$', r'$p(D_{t=1 hr},T_{t=1 hr}) \quad X \quad T_{ever}$',\
               r'$p(D_{t=1 hr},T_{t=1 hr}) \quad X \quad D|T_{ever}$', r'$p(D_{t=1 hr},T_{t=1 hr}) \quad X \quad MortStatus$']
yLabel=[r'$p(T_{t=1 hr})$', r'$p(T_{t=1 hr})$',\
       r'$p(T_{t=1 hr})$', r'$p(D_{t=1 hr},T_{t=1 hr})$',\
       r'$p(D_{t=1 hr},T_{t=1 hr}) $', r'$p(D_{t=1 hr},T_{t=1 hr})$']
xLabel=[r'$T_{ever}$', r'$D|T_{ever}$',\
       r'$MortStatus$', r'$T_{ever}$',\
       r'$D|T_{ever}$', r'$MortStatus$']
df_metrics=getLongerOutcomesMetrics(modelNames, tasks, estNames, outcome_D, title, xLabel, yLabel,
df_metrics, df_ids_test, dict_df_labels['LGBM'], dict_df_probs['LGBM'], df_labels_wo_cutoff, 
                                    df_ids_test_wo_cutoff, outcome_D_name=outcome_D_name)

In [ ]:
modelNames=['LGBM']
tasks=['T', 'D_pseudo']
estNames=[r'$p(T_{t=1hr}=1|X)$', r'Pseudo Labels']
outcome_D='D|T'
outcome_D_name='D|T'
title=[r'$p(T_{t=1 hr}) \quad X \quad T_{ever}$', r'$p(T_{t=1 hr}) \quad X \quad D|T_{ever}$',\
               r'$p(T_{t=1 hr}) \quad X \quad MortStatus$', r'$p(D_{t=1 hr},T_{t=1 hr}) \quad X \quad T_{ever}$',\
               r'$p(D_{t=1 hr},T_{t=1 hr}) \quad X \quad D|T_{ever}$', r'$p(D_{t=1 hr},T_{t=1 hr}) \quad X \quad MortStatus$']
yLabel=[r'$p(T_{t=1 hr})$', r'$p(T_{t=1 hr})$',\
       r'$p(T_{t=1 hr})$', r'$p(D_{t=1 hr},T_{t=1 hr})$',\
       r'$p(D_{t=1 hr},T_{t=1 hr}) $', r'$p(D_{t=1 hr},T_{t=1 hr})$']
xLabel=[r'$T_{ever}$', r'$D|T_{ever}$',\
       r'$MortStatus$', r'$T_{ever}$',\
       r'$D|T_{ever}$', r'$MortStatus$']
df_metrics=getLongerOutcomesMetrics(modelNames, tasks, estNames, outcome_D, title, xLabel, yLabel,
df_metrics, df_ids_test, dict_df_labels['LGBM'], dict_df_probs['LGBM'], df_labels_wo_cutoff, 
                                    df_ids_test_wo_cutoff, outcome_D_name=outcome_D_name)

In [ ]:
df_metrics

In [ ]:
def plotLongerOutcomes(df_metrics, plot_title, subsetBy, subfig, selectrowNames, figsize=(15,7)):
    tab_colors = ['tab:blue', 'tab:orange']
    ax1=subfig.add_subplot(1,2,1)
    df_temp = df_metrics[df_metrics['colName']==subsetBy][['AUC','modelName','rowName']]
    df_temp.index = df_temp.rowName
    df_temp = df_temp.drop_duplicates()
    sns.barplot(data=df_temp, x='rowName', y='AUC', hue='rowName', ax=ax1, dodge=False, palette=tab_colors)
    ax1.get_legend().remove()
#     ax1.set_xticklabels(selectrowNames, rotation=90,fontsize=20)
    ax1.set_xticklabels([])
    ax1.get_xaxis().set_visible(False)
    ax1.set_xlabel('')
    ylabel = ax1.get_ylabel()
    ax1.set_ylabel(ylabel, fontsize=20)
    ax1.tick_params(axis='y', labelsize=15)
    ax1.yaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%.2f'))


    ax2=subfig.add_subplot(1,2,2)
    df_temp = df_metrics[df_metrics['colName']==subsetBy][['PR','modelName','rowName']]
    df_temp.index = df_temp.rowName
    df_temp = df_temp.drop_duplicates()
    sns.barplot(data=df_temp, x='rowName', y='PR', hue='rowName', ax=ax2, dodge=False,palette=tab_colors)
    ax2.get_legend().remove()
#     ax2.set_xticklabels(selectrowNames, rotation=90,fontsize=20)
    ax2.set_xticklabels([])
    ax2.get_xaxis().set_visible(False)
    ax2.set_xlabel('')
    ylabel = ax2.get_ylabel()
    ax2.yaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%.2f'))
    ax2.set_ylabel(ylabel, fontsize=20)
    ax2.tick_params(axis='y', labelsize=15)
    
#     fig.legend(loc='center right',bbox_to_anchor=(1, 1.3), fontsize=15)
#     subfig.tight_layout()
    subfig.suptitle(plot_title, fontsize=20)
    subfig.subplots_adjust(top=0.9, wspace = 0.5)
    return subfig, ax1.get_legend()

In [ ]:
figsize=(20,5)
fig=plt.figure(figsize=figsize)
subfig=fig.subfigures(nrows=1, ncols=3)

# Subplot 1
plot_title=r'$predicting \quad T_{t>1hr}$'
subsetBy='T'
selectModels=['LGBM']
figsize=(7,4)
df_metrics_T=df_metrics.copy()
df_metrics_T=df_metrics_T.loc[df_metrics_T['modelName'].isin(selectModels)]
selectrowNames=[r'$p(T_{t=1hr}=1|X)$', r'$p(D_{t=1hr}=1|T_{t=1hr}=1,X)$', r'$p(D_{t=1hr}=1,T_{t=1hr}=1|X)$', r'$IPW_{t=1hr}$', r'Pseudo Labels']
selectcolNames=['T', 'D|T', 'Mortality']
# df_metrics_T['colName'] = df_metrics_T['colName'].apply(lambda x: x in selectTasks)
df_metrics_T=df_metrics_T.loc[df_metrics_T['rowName'].isin(selectrowNames)]
df_metrics_T=df_metrics_T.loc[df_metrics_T['colName'].isin(selectcolNames)]

subfig[0], _ =plotLongerOutcomes(df_metrics_T, plot_title, subsetBy, subfig[0], selectrowNames, figsize=figsize)

# Subplot 2
plot_title=r'$predicting \quad D|T_{t>1hr}$'
subsetBy='D|T'
selectModels=['LGBM']
figsize=(7,4)
subfig[1], _ =plotLongerOutcomes(df_metrics_T, plot_title, subsetBy, subfig[1], selectrowNames, figsize=figsize)

# Subplot 3
plot_title=r'$predicting \quad Mortality$'
subsetBy='Mortality'
selectModels=['LGBM']
figsize=(7,4)
subfig[2], lgnd =plotLongerOutcomes(df_metrics_T, plot_title, subsetBy, subfig[2], selectrowNames, figsize=figsize)
plt.legend(handles=lgnd, loc='center', ncol=2, bbox_to_anchor=(-3.6, -0.2), fontsize=20)
plt.show()

In [ ]:
df_metrics_T=df_metrics_T.drop(columns=['BalancedAcc'])

In [ ]:
df_metrics_T.drop_duplicates(inplace=True)
mName='LGBM_w_feat_v2_cutoff_T'
path='_depth_7_subsample-data_1.0_subsample-feat_1.0'
df_metrics_T_path=osp.join(os.environ.get('USER_PATH'),'HIRID_Repo', 'logs', 'benchmark_exp', str(mName), str(path))
df_metrics_T.to_csv(df_metrics_T_path+"/df_metrics_longTerm.csv", index=False)

In [ ]:
df_metrics_T